# IT Academy - Data Science amb Python
## Tasca 7: Estructura de control

###  [Github Registre de Logs](https://github.com/jesussantana/Registre_de_logs)

###  Exercici 3
- Geolocalitza les IP's. Aqui tens una pagina de interes:
  - [IP2Locattion](https://blog.ip2location.com/knowledge-base/how-to-add-a-sub-account-in-ip2location/)

In [81]:
import numpy as np
import pandas as pd
import re
import time
from datetime import datetime
from datetime import timedelta

import json
from ip2geotools.databases.noncommercial import DbIpCity

In [82]:
path = '../Data/Ips_export.csv'

Ips_raw = pd.read_csv(path, sep= 'delimiter', engine='python')

Ips_raw.head()

,IP
0,127.0.0.1
1,127.0.0.1
2,127.0.0.1
3,127.0.0.1
4,127.0.0.1


In [83]:
Ips_raw.tail()

,IP
261868,5.255.253.53
261869,74.86.158.107
261870,127.0.0.1
261871,127.0.0.1
261872,127.0.0.1


In [84]:
Ips_copy = Ips_raw.copy()
ips = Ips_copy.IP.unique()

In [85]:
len(Ips_copy.IP.unique())

2921

In [5]:
# Function for extract Locations

localhost = "127.0.0.1"

def extract_location(ip):
    
    try:
        if ip == localhost:
            return ('Local', 'Local')

        else:
            response = DbIpCity.get(ip, api_key = 'free').to_json()    
            data = json.loads(response)
            return data['city'], data['country']
    
    except:
        return np.nan

In [96]:
# Extract Location Test

extract_location(Ips_copy.IP[50000])

nan

In [7]:
# List of locations for unique Ips
start = time.monotonic()

locations = []

for item in ips:
    
    locations.append(extract_location(item))  

end = time.monotonic()

print(timedelta(seconds = end - start))

0:12:44.235354


In [48]:
locations[1]

('Mountain View', 'US')

In [73]:
ips = pd.Series(ips)
locations = pd.Series(locations)
frame_data= {'IP': ips, 'Locations': locations}
Ips_Locations = pd.DataFrame(frame_data)

In [74]:
Ips_Locations.head()

,IP,Locations
0,127.0.0.1,"(Local, Local)"
1,66.249.76.216,"(Mountain View, US)"
2,5.255.253.53,"(Moscow (Tsentralnyy administrativnyy okrug), RU)"
3,157.55.35.112,"(Newark, US)"
4,193.111.141.116,"(Amsterdam (Nieuwmarkt en Lastage), NL)"


In [75]:
Ips_Locations.tail()

,IP,Locations
2916,46.222.83.180,NaN
2917,79.155.78.117,NaN
2918,217.174.248.179,NaN
2919,188.135.173.80,NaN
2920,206.198.5.33,NaN


In [88]:
start = time.monotonic()

df = Ips_copy.merge(Ips_Locations, how='inner')
   
end = time.monotonic()

print(timedelta(seconds = end - start))

0:00:00.042559


In [89]:
df

,IP,Locations
0,127.0.0.1,"(Local, Local)"
1,127.0.0.1,"(Local, Local)"
2,127.0.0.1,"(Local, Local)"
3,127.0.0.1,"(Local, Local)"
4,127.0.0.1,"(Local, Local)"
...,...,...
261868,217.174.248.179,NaN
261869,217.174.248.179,NaN
261870,188.135.173.80,NaN
261871,188.135.173.80,NaN


In [ ]:
# Compare ips & add location
'''start = time.monotonic()

IP_locations = []
count = 1

for index, row in Ips_copy.iterrows():
    
    # iterate the two lists
    for a, b in zip(ips, locations):
        
        # Compare & add
        if row.IP == a:
            IP_locations.append(b)  
            print(b, count)
            count += 1

end = time.monotonic()

print(timedelta(seconds = end - start))'''

In [91]:
Locations_export= df.Locations.copy()

Locations_export.replace('', 'null', inplace = True)

Locations_export.to_csv('../Data/Locations_export.csv', index = False)